# Notebook Overview

TLDR; Scrape Data, Create CSV file

Scrape Data:
* Request HTML from [My Student Halls](https://www.mystudenthalls.com/) site
* Scrape for Property, Owner/Operator, Address, Link URL
* Scrape for 1 x Type, Rent (p/w vs pcm), weeks
* Add data for 1 to master df
* Scrape for N x (Type, Rent (p/w vs pcm), weeks)
* Add data for N to master df
* Save as CSV
* Scrape for Area (sqm vs sqft)

# Import Libraries

In [1]:
# DataFrame
import pandas as pd

# Webscraping
from bs4 import BeautifulSoup as bs
import requests

# Creation of DataFrames

In [2]:
# Initial DataFrame to scrape the URLs
df_urls = pd.DataFrame(columns=['Property', 'url'])

# Column headings to be used in DataFrame
cols = ['Property', 'Owner / Operator', 'Type', 'Rent pw',
        'Rent pcm', 'Rent pa', 'Weeks', 'Area sqm', 'Area sqft',
        '£psf pa', 'Address', 'url']

# Master DataFrame to be populated
df = pd.DataFrame(columns=cols)

# Temporary DataFrame to be populated, added to Master DF then cleared
df_temp = pd.DataFrame(columns=cols)

# Which University?

In [3]:
# To be filled in by user
university = input()
url_home = 'https://www.mystudenthalls.com/?s=' + university

Leeds


# Web Scrape

## DF_URL (Scrape Property Names & associated URLs)

In [4]:
# Make a get request to retrieve the page
html_page = requests.get(url_home)

# Pass the page contents to beautiful soup for parsing
soup = bs(html_page.text, 'html.parser')

### Scrape all property names at the university

In [5]:
# Retreive all property names
all_properties = soup.find_all('div', class_='listing-detail')

In [6]:
# List for all the property names
property_names = []
for property_ in list(range(len(all_properties))):
    # Scrape the name of the nth property
    property_name = all_properties[property_].find("h2").get_text()
    # Add property name to the list of property names
    property_names.append(property_name)
    
# Add list of all property names to DataFrame
df_urls['Property'] = property_names

### Scrape all URLs for the properties

In [7]:
# Create a list of the properties' url links
links = []
for property_ in list(range(len(all_properties))):
    link = all_properties[property_].h2.a['href']
    links.append(link)
    
# Add List of properties' url links to df_url
df_urls['url'] = links

# Preview DataFrame
df_urls.head(2)

,Property,url
0,The Refinery,https://www.mystudenthalls.com/student-accommo...
1,The Foundry,https://www.mystudenthalls.com/student-accommo...


## DF_Temp (temporary dataframe to collect all data for 1 property)

In [8]:
# n = 0

# # nth property
# url = df_urls['url'][n]

# # Make a get request to retrieve the page
# html_page = requests.get(url)

# # Pass the page contents to beautiful soup for parsing
# soup = bs(html_page.text, 'html.parser')

### Type

In [9]:
# # List for all the types of room at this property
# rooms = soup.findAll('p', attrs={'class':'room_title'})

# types = []
# for room_num in list(range(len(rooms))):
#     types.append(rooms[room_num].text)
    
# # Add Room Type to df_temp
# df_temp['Type'] = types

In [8]:
def func_type(soup):
    # List for all the types of room at this property
    rooms = soup.findAll('p', attrs={'class':'room_title'})

    types = []
    for room_num in list(range(len(rooms))):
        types.append(rooms[room_num].text)

    return types

In [23]:
# Add Room Type to df_temp
# df_temp['Type'] = func_type(soup)

### Rent PA

In [10]:
# # HTML that contains info we need for Rent PA
# rents_pa = soup.findAll('li', attrs={'class':'tenancy'})

# # Blank list to gather the different rents PA
# rent_pa = []

# # For each room, add the rent price to the blank list above
# for rent_pa_num in list(range(len(rents_pa))):
#     room_rent = rents_pa[rent_pa_num].text
#     rent_pa.append(int(room_rent[13:-3].replace(',','')))

# # Add Rent PA to df_temp
# df_temp['Rent pa'] = rent_pa

In [9]:
def func_rent_pa(soup):
    # HTML that contains info we need for Rent PA
    rents_pa = soup.findAll('li', attrs={'class':'tenancy'})

    # Blank list to gather the different rents PA
    rent_pa = []

    # For each room, add the rent price to the blank list above
    for rent_pa_num in list(range(len(rents_pa))):
        room_rent = rents_pa[rent_pa_num].text
        rent_pa.append(int(room_rent[13:-3].replace(',','')))
    
    return rent_pa

### Weeks

In [11]:
# # HTML that contains info we need for Weeks
# weeks_data = soup.findAll('li', attrs={'class':'availability'})

# # List of number of weeks room is available for in the year
# weeks = []
# for n in list(range(len(weeks_data)))[1::2]: #availablity tag also returns good/limited info of room, so we only want every other one
#     # add only the number of weeks to the list
#     weeks.append(int(weeks_data[n].text[7:9]))

# # Add number of weeks to the dataframe    
# df_temp['Weeks'] = weeks

In [10]:
def func_weeks(soup):
    # HTML that contains info we need for Weeks
    weeks_data = soup.findAll('li', attrs={'class':'availability'})

    # List of number of weeks room is available for in the year
    weeks = []
    for n in list(range(len(weeks_data)))[1::2]: #availablity tag also returns good/limited info of room, so we only want every other one
        # add only the number of weeks to the list
        weeks.append(int(weeks_data[n].text[7:9]))
    
    return weeks

### Rent PW

In [12]:
# df_temp['Rent pw'] = df_temp['Rent pa'] / df_temp['Weeks']

### Rent PCM

In [13]:
# df_temp['Rent pcm'] = df_temp['Rent pa'] / 12

### Address

In [14]:
# HTML that contains info we need for Address
address_data = soup.findAll('div', attrs={'class':'inner'})

df_temp['Address'] = address_data[5].h3.text[9:]

### Owner/Operator

In [15]:
# # Scrape the Owner/Operator name
# owner = soup.findAll('aside', attrs={'id':'sidebar-detail'})
# owner = owner[0].h2.text

# # Add Owner/Operator name to df_temp
# df_temp['Owner / Operator'] = owner

### URL

In [16]:
# df_temp['url'] = url

### Property

In [17]:
# df_temp['Property'] = df_urls['Property'][n]

## Function to populate df_temp

In [11]:
def func_build_df_temp(n, soup):
    '''Populate the df_temp dataframe
    n: the property number as it appears in the list from the url_home
    soup: beautiful soup of the nth property's url, to be parsed'''

    # Column headings to be used in DataFrame
    cols = ['Property', 'Owner / Operator', 'Type', 'Rent pw',
        'Rent pcm', 'Rent pa', 'Weeks', 'Area sqm', 'Area sqft',
        '£psf pa', 'Address', 'url']

    # Temporary DataFrame to be populated, added to Master DF then cleared
    df_temp = pd.DataFrame(columns=cols)
    
    # Type
    df_temp['Type'] = func_type(soup)
    # RentPA
    df_temp['Rent pa'] = func_rent_pa(soup)
    # Weeks
    df_temp['Weeks'] = func_weeks(soup)
    # RentPW
    df_temp['Rent pw'] = df_temp['Rent pa'] / df_temp['Weeks']
    # RentPCM
    df_temp['Rent pcm'] = df_temp['Rent pa'] / 12
    # Address
    # HTML that contains info we need for Address
    df_temp['Address'] = soup.findAll('div',attrs={'class':'inner'})[5].h3.text[9:]
    # Owner/Operator
    df_temp['Owner / Operator'] = soup.findAll('aside', attrs={'id':'sidebar-detail'})[0].h2.text
    # URL
    df_temp['url'] = df_urls['url'][n]
    # Property
    df_temp['Property'] = df_urls['Property'][n]
    
    return df_temp

In [12]:
for n in list(range(len(df_urls))):
    # Make a get request to retrieve the nth property's url page
    html_page = requests.get(df_urls['url'][n])

    # Pass the page contents to beautiful soup for parsing
    soup = bs(html_page.text, 'html.parser')
    
    # Build df_temp of nth property
    df_temp = func_build_df_temp(n, soup)
    
    # Concatenate (add) df_temp to df
    df = pd.concat([df, df_temp])        

    # Clear df_temp
    df_temp = pd.DataFrame(columns=cols)

In [13]:
df

,Property,Owner / Operator,Type,Rent pw,Rent pcm,Rent pa,Weeks,Area sqm,Area sqft,£psf pa,Address,url
0,The Refinery,Fresh,Bronze En Suite,169.0,619.666667,7436,44,NaN,NaN,NaN,"Bingley Street, Leeds, Leeds, LS3 1BZ",https://www.mystudenthalls.com/student-accommo...
1,The Refinery,Fresh,Bronze En Suite,164.0,697.000000,8364,51,NaN,NaN,NaN,"Bingley Street, Leeds, Leeds, LS3 1BZ",https://www.mystudenthalls.com/student-accommo...
2,The Refinery,Fresh,Silver En Suite,171.0,627.000000,7524,44,NaN,NaN,NaN,"Bingley Street, Leeds, Leeds, LS3 1BZ",https://www.mystudenthalls.com/student-accommo...
3,The Refinery,Fresh,Silver En Suite,166.0,705.500000,8466,51,NaN,NaN,NaN,"Bingley Street, Leeds, Leeds, LS3 1BZ",https://www.mystudenthalls.com/student-accommo...
4,The Refinery,Fresh,Gold En Suite,174.0,638.000000,7656,44,NaN,NaN,NaN,"Bingley Street, Leeds, Leeds, LS3 1BZ",https://www.mystudenthalls.com/student-accommo...
...,...,...,...,...,...,...,...,...,...,...,...,...
0,Oxley Residences,University of Leeds,Standard Room,92.0,322.000000,3864,42,NaN,NaN,NaN,"Oxley Residences, Weetwood Lane, Leeds, LS16 8HL",https://www.mystudenthalls.com/student-accommo...
1,Oxley Residences,University of Leeds,En-suite,117.0,409.500000,4914,42,NaN,NaN,NaN,"Oxley Residences, Weetwood Lane, Leeds, LS16 8HL",https://www.mystudenthalls.com/student-accommo...
0,Leeds University Smaller Residences,University of Leeds,Houses/ Flats - Shared bathroom,91.0,310.916667,3731,41,NaN,NaN,NaN,"Variable: See above, Leeds, LS6 2BZ",https://www.mystudenthalls.com/student-accommo...
1,Leeds University Smaller Residences,University of Leeds,Houses/ Flats - Ensuite,126.0,430.500000,5166,41,NaN,NaN,NaN,"Variable: See above, Leeds, LS6 2BZ",https://www.mystudenthalls.com/student-accommo...


## Add df_temp to df and clear df_temp